In [ ]:
!pip install ray[rllib] tensorflow
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

In [7]:
import argparse
import gymnasium as gym
import os

import numpy as np
import ray
from ray.air import Checkpoint
from ray.air.config import RunConfig
from ray.train.rl.rl_predictor import RLPredictor
from ray.train.rl.rl_trainer import RLTrainer
from ray.air.config import ScalingConfig
from ray.air.result import Result
from ray.rllib.algorithms.bc import BC
from ray.tune.tuner import Tuner

In [8]:
def train_rl_ppo_online(num_workers: int, use_gpu: bool = False) -> Result:
    print("Starting online training")
    trainer = RLTrainer(
        run_config=RunConfig(stop={"training_iteration": 5}),
        scaling_config=ScalingConfig(num_workers=num_workers, use_gpu=use_gpu),
        algorithm="PPO",
        config={
            "env": "CartPole-v1",
            "framework": "tf",
        },
    )
    # Todo (krfricke/xwjiang): Enable checkpoint config in RunConfig
    # result = trainer.fit()
    tuner = Tuner(
        trainer,
        _tuner_kwargs={"checkpoint_at_end": True},
    )
    result = tuner.fit()[0]
    return result

In [9]:
def evaluate_using_checkpoint(checkpoint: Checkpoint, num_episodes) -> list:
    predictor = RLPredictor.from_checkpoint(checkpoint)

    env = gym.make("CartPole-v1")

    rewards = []
    for i in range(num_episodes):
        obs = env.reset()
        reward = 0.0
        done = False
        while not done:
            action = predictor.predict(np.array([obs]))
            obs, r, done, _ = env.step(action[0])
            reward += r
        rewards.append(reward)

    return rewards

In [12]:
result = train_rl_ppo_online(num_workers=2, use_gpu=False)

Starting online training


2023-02-21 23:55:44,189	INFO worker.py:1538 -- Started a local Ray instance.


2023-02-21 23:55:52,615	INFO algorithm_config.py:2503 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2023-02-21 23:55:52,618	INFO algorithm_config.py:2503 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
(pid=846107) /run/media/nishant/Data/Work/uni/thesis/repo/env/lib/python3.9/site-packages/tensorflow/python/framework/dtypes.py:246: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
(pid=846107)   np.bool8: (False, True),
(pid=846107) /run/media/nishant/Data/Work/uni/thesis/repo/env/lib/python3.9/site-packages/pkg_resources/__init__.py:2804:

Trial name,trial_id
AIRPPO_e1b56_00000,e1b56_00000


2023-02-21 23:58:41,020	ERROR ray_trial_executor.py:118 -- An exception occurred when trying to stop the Ray actor:Traceback (most recent call last):
  File "/run/media/nishant/Data/Work/uni/thesis/repo/env/lib/python3.9/site-packages/ray/tune/execution/ray_trial_executor.py", line 109, in _post_stop_cleanup
    ray.get(future, timeout=timeout)
  File "/run/media/nishant/Data/Work/uni/thesis/repo/env/lib/python3.9/site-packages/ray/_private/client_mode_hook.py", line 105, in wrapper
    return func(*args, **kwargs)
  File "/run/media/nishant/Data/Work/uni/thesis/repo/env/lib/python3.9/site-packages/ray/_private/worker.py", line 2311, in get
    raise value
ray.exceptions.RayActorError: The actor died because of an error raised in its creation task, ray::AIRRLTrainer.__init__() (pid=846107, ip=192.168.10.111, repr=AIRPPO)
  File "/run/media/nishant/Data/Work/uni/thesis/repo/env/lib/python3.9/site-packages/ray/train/rl/rl_trainer.py", line 209, in __init__
    super(AIRRLTrainer, self)._

In [15]:
result

Result(metrics={'trial_id': 'e1b56_00000'}, error=TuneError('Failure # 1 (occurred at 2023-02-21_23-58-40)\nTraceback (most recent call last):\n  File "/run/media/nishant/Data/Work/uni/thesis/repo/env/lib/python3.9/site-packages/ray/tune/execution/ray_trial_executor.py", line 1070, in get_next_executor_event\n    future_result = ray.get(ready_future)\n  File "/run/media/nishant/Data/Work/uni/thesis/repo/env/lib/python3.9/site-packages/ray/_private/client_mode_hook.py", line 105, in wrapper\n    return func(*args, **kwargs)\n  File "/run/media/nishant/Data/Work/uni/thesis/repo/env/lib/python3.9/site-packages/ray/_private/worker.py", line 2311, in get\n    raise value\nray.exceptions.RayActorError: The actor died because of an error raised in its creation task, \x1bray::AIRRLTrainer.__init__()\x1b (pid=846107, ip=192.168.10.111, repr=AIRPPO)\n  File "/run/media/nishant/Data/Work/uni/thesis/repo/env/lib/python3.9/site-packages/ray/train/rl/rl_trainer.py", line 209, in __init__\n    super(

In [13]:
num_eval_episodes = 3

rewards = evaluate_using_checkpoint(result.checkpoint, num_episodes=num_eval_episodes)
print(f"Average reward over {num_eval_episodes} episodes: " f"{np.mean(rewards)}")

AttributeError: 'NoneType' object has no attribute '_local_path'